In [1]:
import jsonlines
import pickle
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter

In [60]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# How to debug

Creating an jsonline reader and iterating line by line

In [63]:
reader = jsonlines.open('data/data.jsonl')
obj = reader.read()
text = word_tokenize(obj['casebody']['data']['opinions'][0]['text'])

In [65]:
for token in Counter(text):
    print(token)

Facilities
allege
731
whether
existing
2—2
Adm.
remedy
nondiscretionary
however
shall
follows
Securities
875
v.
***
Krilich
The
Procedures
Bank
issue
2097
5/1
failing
Caruth
defined
(
Thompson
For
controlled
Lane
right
Civil
Beginning
100/1—
504.80
trial
Co.
1973
it
Will
Illinois
contested
Trust
such
does
follow
defendant
Department
615
granted
Kenneth
5
complaint
determined
ANALYSIS
credits
established
855
100/1
2
Correctional
8—8
eff
Unified
effect.
disciplinary
reasons
Briley
Dismiss
Weiss
He
If
2000
no
apply
West
2003
We
administrative
if
did
comply
argued
Central
upon
18
filed
grievance
County
failure
those
cases
Michael
covered
disciplined
in
rights
Administrative
facts
was
cases.
adjudicatory
perform
used
See
CONCLUSION
§
power
In
On
entitled
Because
June
[
err
standard
R.
Affirmed
Southern
violated
agency
when
conduct
333
sufficiency
&
now
5/2
JUSTICE
submitted
563
uncontested
8—7
775
Follow
3d
affirm
A.
Action
reference
law
inter
public
3
statutory
alia
77
relief
consistent
pr

# How to process

In [2]:
def extractJurisdiction(obj):
    if len(obj['casebody']['data']['opinions']) > 0:
        return(obj['casebody']['data']['opinions'][0]['text'])
    return('')

In [12]:
word_freq_dict = {}
doc_freq_dict = {}

def extractDocWordFreq(obj, word_freq_dict, doc_freq_dict):
    if len(obj['casebody']['data']['opinions']) > 0:
        text = word_tokenize(obj['casebody']['data']['opinions'][0]['text'])
        counter_text = Counter(text)
        for token in counter_text:
            if not(token in word_freq_dict):
                word_freq_dict[token] = counter_text[token]
            else:
                word_freq_dict[token] += counter_text[token]
            if not(token in doc_freq_dict):
                doc_freq_dict[token] = 1
            else:
                doc_freq_dict[token] += 1
    
    return(word_freq_dict, doc_freq_dict)

In [14]:
output = []
reader = jsonlines.open('data/data.jsonl')
obj = reader.read()
i = 0

import time
start_time = time.time()

while obj != None:
    #word_freq_dict, doc_freq_dict = extractDocWordFreq(obj, word_freq_dict, doc_freq_dict)
    obj = reader.read()
    i += 1
    if i % 5000 == 0:
        print(i)
        print(time.time() - start_time)
        start_time = time.time()
        #pickle.dump(word_freq_dict, open('output/word_freq_dict_' + str(i) + '.p', "wb" ))
        #pickle.dump(doc_freq_dict, open('output/doc_freq_dict_' + str(i) + '.p', "wb" ))

5000
1.1254680156707764
10000
0.9120457172393799
15000
1.0338215827941895
20000
0.8731708526611328
25000
0.7708120346069336
30000
0.7425713539123535
35000
0.8550662994384766
40000
0.8814551830291748
45000
1.0030097961425781
50000
0.7663078308105469
55000
0.604424238204956
60000
0.7190289497375488
65000
0.7073104381561279
70000
0.7773416042327881
75000
1.1520733833312988
80000
0.960578441619873
85000
1.0527653694152832
90000
1.021216630935669
95000
0.8720707893371582
100000
0.6423850059509277
105000
1.237703800201416
110000
1.0700099468231201
115000
1.5655601024627686
120000
0.9012079238891602
125000
0.5747365951538086
130000
0.8858492374420166
135000
0.708268404006958
140000
0.7808315753936768
145000
0.7648308277130127
150000
0.630570650100708
155000
0.4921088218688965
160000
0.7509787082672119
165000
0.726931095123291
170000
0.8611361980438232
175000
0.7633371353149414
180000
0.899616003036499


EOFError: 

In [78]:
output = []
reader = jsonlines.open('data/data.jsonl')
obj = reader.read()
i = 0

import time
start_time = time.time()

while obj != None:
    output = output + [extractJurisdiction(obj)]
    obj = reader.read()
    i += 1
    if i % 50000 == 0:
        pickle.dump(output, open('output/text_' + str(i) + '.p', "wb" ))
        print(i)
        print(time.time() - start_time)
        start_time = time.time()        

50000
123.26157855987549


KeyboardInterrupt: 

In [15]:
doc_freq = pickle.load(open('output/doc_freq_dict_180000.p', "rb"))
word_freq = pickle.load(open('output/word_freq_dict_180000.p', "rb"))

In [18]:
doc_freq_list = [[i, doc_freq[i]] for i in doc_freq]

In [23]:
import pandas as pd 
df = pd.DataFrame(doc_freq_list)
df.to_csv("doc_freq_dict.csv", index=False, header=True)

<Text: PRESIDING JUSTICE McDADE
delivered the opinion of the court:
Michael Ratliff-El filed a complaint for mandamus relief (735 ILCS 5/14 — 101 et seq. (West 2000)) naming Kenneth R. Briley, warden of the Stateville Correctional Center, as the defendant. The trial court granted the defendant’s motion to dismiss under section 2 — 615 of the Code of Civil Procedure (735 ILCS 5/2 — 615 (West 2000)). On appeal, Ratliff-El argues that the trial court erred by dismissing his complaint because it stated a cause of action for mandamus relief. We affirm.
BACKGROUND
Ratliff-El is a prisoner at Stateville. In his complaint, Ratliff-El contended that the defendant had a clear duty to follow the Illinois Administrative Procedure Act (Act) (5 ILCS 100/1 — 1 et seq. (West 2000)) during adjustment committee and grievance proceedings at Stateville. As a result of such proceedings, Ratliff-El had been disciplined with revocation of good-time credits. Ratliff-El submitted that he was entitled to mandam